# This is Jeopardy!

#### Overview

This project is slightly different than others you have encountered thus far. Instead of a step-by-step tutorial, this project contains a series of open-ended requirements which describe the project you'll be building. There are many possible ways to correctly fulfill all of these requirements, and you should expect to use the internet, Codecademy, and/or other resources when you encounter a problem that you cannot easily solve.

#### Project Goals

You will work to write several functions that investigate a dataset of _Jeopardy!_ questions and answers. Filter the dataset for topics that you're interested in, compute the average difficulty of those questions, and train to become the next Jeopardy champion!

## Prerequisites

In order to complete this project, you should have completed the Pandas lessons in the <a href="https://www.codecademy.com/learn/paths/analyze-data-with-python">Analyze Data with Python Skill Path</a>. You can also find those lessons in the <a href="https://www.codecademy.com/learn/data-processing-pandas">Data Analysis with Pandas course</a> or the <a href="https://www.codecademy.com/learn/paths/data-science/">Data Scientist Career Path</a>.

Finally, the <a href="https://www.codecademy.com/learn/practical-data-cleaning">Practical Data Cleaning</a> course may also be helpful.

## Project Requirements

1. We've provided a csv file containing data about the game show _Jeopardy!_ in a file named `jeopardy.csv`. Load the data into a DataFrame and investigate its contents. Try to print out specific columns.

   Note that in order to make this project as "real-world" as possible, we haven't modified the data at all - we're giving it to you exactly how we found it. As a result, this data isn't as "clean" as the datasets you normally find on Codecademy. More specifically, there's something odd about the column names. After you figure out the problem with the column names, you may want to rename them to make your life easier for the rest of the project.
   
   In order to display the full contents of a column, we've added this line of code for you:
   
   ```py
   pd.set_option('display.max_colwidth', None)
   ```

In [1]:
import pandas as pd
import numpy as np
import string
import random
from time import sleep
pd.set_option('display.max_colwidth', None)

jeopardy = pd.read_csv('jeopardy.csv')
jeopardy.set_axis(['show_number', 'air_date', 'round', 'category', 'value', 'question', 'answer'],\
                  axis='columns', inplace = True)

jeopardy['qlist'] = [set(words) for words in
    jeopardy['question']
    .str.lower()
    .str.replace("'", " ")
    .str.replace('[{0}]*'.format(string.punctuation), '')
    .str.strip()
    .str.split()
]

jeopardy.value = jeopardy.value.apply(lambda x: int(x.replace("$", "").replace(",", "")) if x != 'None' else 0).astype('int64')
jeopardy.answer.fillna('Null', inplace=True)
jeopardy.air_date = jeopardy.air_date.astype('datetime64')

print(jeopardy.info())
print(jeopardy.air_date.min())

C:\Users\cowha\AppData\Local\Temp/ipykernel_11520/1387900380.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  jeopardy['question']


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216930 entries, 0 to 216929
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   show_number  216930 non-null  int64         
 1   air_date     216930 non-null  datetime64[ns]
 2   round        216930 non-null  object        
 3   category     216930 non-null  object        
 4   value        216930 non-null  int64         
 5   question     216930 non-null  object        
 6   answer       216930 non-null  object        
 7   qlist        216930 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 13.2+ MB
None
1984-09-10 00:00:00


2. Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list `["King", "England"]` was passed to our function, the function returned a DataFrame of 49 rows. Every row had the strings `"King"` and `"England"` somewhere in its `" Question"`.

   Test your function by printing out the column containing the question of each row of the dataset.

In [2]:
def filter_data(data, words):
    filter = lambda x: all(word in x for word in words)
    return data.loc[data["question"].apply(filter)].reset_index()
    
print(filter_data(jeopardy, ['King', 'England']).question)

0                                                                                                                                                                                                                                                                     Both England's King George V & FDR put their stamp of approval on this "King of Hobbies"
1                                                                                                                                                                                                                                                            This country's King Louis IV was nicknamed "Louis From Overseas" because he was raised in England
2                                                                                                                                                                                                                                                                                 this man and his son rul

3. Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust.

   For example, think about capitalization. We probably want to find questions that contain the word `"King"` or `"king"`.
   
   You may also want to check to make sure you don't find rows that contain substrings of your given words. For example, our function found a question that didn't contain the word `"king"`, however it did contain the word `"viking"` &mdash; it found the `"king"` inside `"viking"`. Note that this also comes with some drawbacks &mdash; you would no longer find questions that contained words like `"England's"`.

In [3]:
def filter_data(data, words):
    filter = lambda x: all(word.lower() in x for word in words)
    return data.loc[data["qlist"].apply(filter)].reset_index()
    
filtered = filter_data(jeopardy, ['King'])

print(filtered.question)

0       <a href="http://www.j-archive.com/media/2004-12-31_DJ_26.mp3">Ripped from today's headlines, he was a turtle king gone mad; Mack was the one good turtle who'd bring him down</a>
1                     <a href="http://www.j-archive.com/media/2004-12-31_DJ_24.mp3">"500 Hats"... 500 ways to die.  On July 4th, this young boy will defy a king... & become a legend</a>
2                                                                                                                     Between 1842 & 1885, he repeatedly revised his "Idylls of the King"
3                                                                                                         Robin Quivers is the radio consort of this self-proclaimed  "King of All Media"
4                                                                                                                A Norman could say, "I'm the king of the motte-and-bailey style of" this
                                                                      

4. We may want to eventually compute aggregate statistics, like `.mean()` on the `" Value"` column. But right now, the values in that column are strings. Convert the`" Value"` column to floats. If you'd like to, you can create a new column with float values.

   Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the "difficulty" of certain topics. For example, what is the average value of questions that contain the word `"King"`?
   
   Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.

In [4]:
print(filtered.value.mean())

803.8715596330276


5. Write a function that returns the count of unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word `"King"`, we could then find all of the unique answers to those questions. The answer "Henry VIII" appeared 55 times and was the most common answer.

In [5]:
def unique_answers(data):
    return data.groupby('answer').question.count().sort_values(ascending=False).reset_index()

print(unique_answers(filtered))

                  answer  question
0             Henry VIII        54
1              Louis XIV        31
2                Solomon        31
3                  David        29
4            Richard III        24
...                  ...       ...
1480               India         1
1481                Inca         1
1482  In (the beginning)         1
1483               Ibsen         1
1484               zlati         1

[1485 rows x 2 columns]


6. Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven't even started looking at yet. Here are some ideas on ways to continue working with this data:

 * Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word `"Computer"` compared to questions from the 2000s?
 * Is there a connection between the round and the category? Are you more likely to find certain categories, like `"Literature"` in Single Jeopardy or Double Jeopardy?
 * Build a system to quiz yourself. Grab random questions, and use the <a href="https://docs.python.org/3/library/functions.html#input">input</a> function to get a response from the user. Check to see if that response was right or wrong.

In [6]:
# def round_count(data, words):
#     rounds = filter_data(data, words)
#     return rounds.groupby('round').question.count().sort_values(ascending=False).reset_index()

# print(round_count(jeopardy, ['jesus']))

def jeopardy_game():
    while True:
        while True:
            selectround = input('Select your round ("Jeopardy!", "Double Jeopardy!", or "Final Jeopardy!"):\n')
            if selectround not in jeopardy['round'].tolist():
                print('Please select a valid round.')
            else: break
        round_filtered = jeopardy.loc[jeopardy['round'] == selectround].category.unique()
        randcat = random.sample(range(len(round_filtered)), 6)
        categories = {str(x+1): round_filtered[randcat[x]] for x in range(6)}
        print('Your categories: ')
        print(categories)
        while True:
            catans = input('Select a category:\n')
            if catans in categories:
                chosen_category = categories[catans]
                print("You've chosen: " + chosen_category)
                break
            else:
                print('Please select your category by typing the number of your chosen category')
        if selectround == 'Jeopardy!' or selectround == 'Double Jeopardy!':
            wager = jeopardy.loc[(jeopardy['round'] == selectround) & (jeopardy.category == chosen_category)]\
                            .value.unique().tolist()
            while True:
                print("Select wager: ", wager,\
                      "\nNote: non-standard wagers may be Daily Double questions, older games, or specialty games.\n")
                wagerchoose = input('Your wager:\n')
                try:
                    wagerchoose = int(wagerchoose)
                    if wagerchoose not in wager:
                        print("Please select a wager from the list")
                    else:
                        questionfiltered = jeopardy.loc[
                                                        (jeopardy['round'] == selectround) &
                                                        (jeopardy.category == chosen_category) &
                                                        (jeopardy.value == wagerchoose)
                                           ].reset_index(drop=True)
                        break
                except ValueError:
                    print("Invalid input, please try again.")
        else:
            while True:
                try:
                    wagerchoose = input("How much would you like to wager?\n")
                    wagerchoose = int(wagerchoose)
                    questionfiltered = jeopardy.loc[
                                                    (jeopardy['round'] == selectround) &
                                                    (jeopardy.category == chosen_category)
                                       ].reset_index(drop=True)
                    break
                except ValueError:
                    print("Invalid input, please try again.")
        #print(questionfiltered)
        questionno = random.randint(0,questionfiltered.index.max())
        print("Here is your clue!\n\n   ", questionfiltered.loc[questionno, 'question'])
        answer = input('What is your guess? Don\'t include "Who is", "What is", etc. ')
        if answer.lower() == questionfiltered.loc[questionno, 'answer'].lower():
            print("That's correct, congratulations! Feel free to add ${} to your fake virtual score."\
                 .format(wagerchoose))
        else:
            print("Sorry, the correct answer was {}. \n\nPlease send ${} to me via paypal or etransfer. Payments not received within 5 business days will be subject to penalties.".format(questionfiltered.loc[questionno, 'answer'], wagerchoose))
        playagain = input("Would you like to play again? Type y or yes to play again, or anything else to exit.\n")
        if playagain.lower() != 'y' and playagain.lower() != 'yes':
            break

jeopardy_game()



Select your round ("Jeopardy!", "Double Jeopardy!", or "Final Jeopardy!"):
Final Jeopardy!
Your categories: 
{'1': 'CLASSIC TV', '2': 'BUSINESS & INDUSTRY', '3': 'U.S. STRUCTURES', '4': 'TAXES', '5': '2002 NEWS', '6': 'U.S. GOVERNMENT'}
Select a category:
4
You've chosen: TAXES
How much would you like to wager?
100000
Here is your clue!

    The tax on this was the subject of a 1794 "Rebellion" & an 1875 "Ring" scandal
What is your guess? Don't include "Who is", "What is", etc. tea
Sorry, the correct answer was Whiskey. 

Please send $100000 to me via paypal or etransfer. Payments not received within 5 business days will be subject to penalties.
Would you like to play again? Type y or yes to play again, or anything else to exit.
c


## Solution

7. Compare your program to our <a href="https://content.codecademy.com/PRO/independent-practice-projects/jeopardy/jeopardy_solution.zip">sample solution code</a> - remember, that your program might look different from ours (and probably will) and that's okay!

8. Great work! Visit <a href="https://discuss.codecademy.com/t/this-is-jeopardy-challenge-project-python-pandas/462365">our forums</a> to compare your project to our sample solution code. You can also learn how to host your own solution on GitHub so you can share it with other learners! Your solution might look different from ours, and that's okay! There are multiple ways to solve these projects, and you'll learn more by seeing others' code.